In [10]:
import numpy as np
from scipy.sparse import diags
from scipy.sparse import identity
from scipy.sparse.linalg import onenormest
from tqdm import tqdm 

In [11]:
def spai(A, m):
    """Perform m step of the SPAI iteration."""

    n = A.shape[0]

    ident = identity(n, format='csr')
    alpha = 2 / onenormest(A @ A.T)
    M = alpha * A

    for _ in tqdm(range(m)):
        C = A @ M
        G = ident - C
        AG = A @ G
        trace = (G.T @ AG).diagonal().sum()
        alpha = trace / np.linalg.norm(AG.data)**2
        M = M + alpha * G

    return M

In [3]:
mesh = np.load("/home/haltmayermarc/FEONet_Stokes/FEONet_Stokes_square_copy/data_ordered/P2x1_ne72_stokes_channel_flow_BC.npz")

In [8]:
matrix = mesh["matrix"]
cond_number = np.linalg.cond(matrix)
cond_number

np.float64(167.32636402645198)

In [22]:
precond = spai(matrix,20000)
new_matrix = matrix @ precond

100%|██████████| 20000/20000 [10:37<00:00, 31.39it/s]


In [25]:
np.save("/home/haltmayermarc/FEONet_Stokes/FEONet_Stokes_square_copy/precond_72_channel_flow.npy", precond)

In [23]:
new_cond_number = np.linalg.cond(new_matrix)
new_cond_number

np.float64(1.4890170268235305)